# Neural Networks

Now that we wrapped out logistic regression for image classification with our hand-written wordset, we hit a classification accuracy of about 97.5%

Thats reasonably good, but pretty much maxes out what we can achieve with a linear model. This time, we'll tackle that same problem with neural networks using a feed-forward neural network with backpropagation.

We'll implement regularized/unregularized versions of the neural network cost function and compute gradients via the backpropagation algorithm.

This isn't for the faint of heart. Best get a cup of coffee for this one...

In [2]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

In [3]:
pwd = os.getcwd()
data = loadmat(pwd + '/asn4/data/ex4data1.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
X = data['X']
y = data['y']
X.shape, y.shape

((5000, 400), (5000, 1))

We're also going to need to one-hot encode our labels. One-hot encoding turns a class label n (out of k classes) into a vector of length k where index n is "hot" (1) and the rest are zero. Essentially, if we have a 4, then we make the 4th index 1, and all the other indices 0, and so on.

Scikit-learn has a built-in utility we can use for this

In [5]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)

print(y_onehot)
print(y_onehot.shape)

[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
(5000, 10)


In [6]:
y_onehot[0, :]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])

The NN we'll build in this exercise has an input layer matching the size of our instance data (400 pixels, so 400 neurons for our input), a hidden layer with 25 units (26 including the bias unit), and an output layer of 10 units, corresponding to the number of classes we have (0-9).

## Cost Function (Unregularized)

The first piece we'll implement is a cost function to evaluate the loss for a given set of network parameters.

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} \bigg[ -y_k^{(i)}log((h_\theta(x^{(i)}))_k) - (1 - y_k^{(i)})log(1 - (h_\theta(x^{(i)}))_k) \bigg] $$

We have the same cost function as before, just with a few tweaks. The main addition here is the K summation. This is; we are doing the cost function, but for *every class*

Remember, the $h_\theta(x^{(i)})$ just means *our hypothesis that results in the kth output*, and we compare this to $ y_k $ (our answer which is for the kth output), all for each training example

In [15]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

## Forward Propagation

Now we'll write our forward propagation algorithm

In [16]:
# we have two theta matrices for each layer. Theta1 is for the input
# layer to calculate the hidden layer, and Theta2 is for the hidden layer
# to calculate the output layer
def forward_propagate(X, theta1, theta2):
    # theta1 = (25, 401)
    # theta2 = (10, 26)
    
    m = X.shape[0]
    
    # add our bias unit to each training example (layer 1)
    a1 = np.insert(X, 0, values=np.ones(m), axis=1) # (5000, 401)
    
    # we combine our input and theta
    z2 = a1 * theta1.T # (5000, 25)
    
    # add our bias unit to each trainnig example (layer 2)
    # we activate our input and theta (which is z2), activating it is
    # running it under the sigmoid function, converting z2 to a2
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1) # (5000, 26)
    
    # we combine our hidden layer with theta, creating z3
    z3 = a2 * theta2.T # (5000, 10)
    
    # final hypothesis, by activating z3 via sigmoid()
    h = sigmoid(z3) #(5000, 10)
    
    return a1, z2, a2, z3, h

In [30]:
def cost(theta_params, input_size, hidden_size, num_labels, X, y, reg_lambda):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    # it is our hidden layer size * input layer size because each hidden
    # layer node is connected to all 400 input nodes, for each hidden
    # node. That's 400 * 25. These Theta parameters are for ALL edges
    # from one layer to another
    
    # (25, 401)
    theta1 = np.matrix(np.reshape(theta_params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    
    # (10, 26)
    theta2 = np.matrix(np.reshape(theta_params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost, vectorized approach
    # because y and h are matrices representing classes, the only important
    # information will be y' * h, which will be a (k x k) matrix (10x10) in
    # this case. That means only their diagonals are important. Think;
    # "show me the error rate of y with class k and h with class k". The
    # only time we have the same classes are when we match them for the 
    # row & col they are in. This means class 1 in y is matched with class 1
    # in h, class 2 for y is matched with class 2 for h, etc... So we
    # only care about the resulting matrix multiplication of the diagonals
    
    # we can get just the diagonal values by using trace(), which does just
    # that. Much more efficient than using for-loops, like this tutorial does!
    J = (1.0 / m) * (np.trace(-y.T * np.log(h)) - np.trace((1 - y).T * np.log(1 - h)))
    return J    

In [27]:
# we'll test this to ensure its working properly. Seeing the output
# from intermediate steps is helpful to understand what's going on

# initial setup
input_size = 400
hidden_size = 25
num_labels = 10
reg_lambda = 1

# randomly initialize a parameter array of the size of the full network's parameters
theta_params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unroll the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(theta_params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(theta_params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25, 401), (10, 26))

In [28]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
print a1.shape, z2.shape, a2.shape, z3.shape, h.shape

(5000, 401) (5000, 25) (5000, 26) (5000, 10) (5000, 10)


The cost function, after computing the hypothesis *h*, applies the cost equation to compute the total error between *y* and *h*

In [32]:
J = cost(theta_params, input_size, hidden_size, num_labels, X, y_onehot, reg_lambda)

## Cost Function (Regularized)

Our next step is adding regularization to our cost function, which adds a penalty term to the cost that scales with the magnitude of the parameters. This is the same as before, just with an added regularization term (looks daunting, but give it a thorough read, it's actually straight forward):

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} \bigg[ -y_k^{(i)}log((h_\theta(x^{(i)}))_k) - (1 - y_k^{(i)})log(1 - (h_\theta(x^{(i)}))_k) \bigg] + \frac{\lambda}{2m} \Bigg[ \sum_{j=1}^{25} \sum_{k=1}^{400} (\Theta_{j,k}^{(1)})^{2} + \sum_{j=1}^{10} \sum_{k=1}^{25} (\Theta_{j,k}^{(2)})^{2} \Bigg]$$

*Breathe*. All this is saying, like above, is, *for every training example, and for every class, calculate the cost with respect to that class*. On top of that, we are saying *for every theta parameter in theta1, penalize it by a certain amount $\lambda$, and do the same for every theta parameter in theta2*

This balances all the theta parameters we'd use for our feed-forward algorithm, which prevents overfitting. The lambda terms are just for keeping our theta parameters in check. Note that the *k* in the last term's summation have nothing to do with classes, and just represent the size dimensions of our theta parameters, which are hardcoded in this equation for this example

In [33]:
# we'll just use our previous cost J and add the regularization terms
J += (float(reg_lambda) / (2 * m)) * (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2)))

## Backpropagation

Now we're ready to implement backpropagation to compute the gradients (this is the learning step). Since the computations required for backprop are superset of those required in the cost function.

THIS is the actual complicated step in feedforward neural nets, so we'll break this into 4 parts:

1) Run your forward propagation algorithm

2) Once you get to the final layer *h*, see how bad you did by comparing your answer for h to our real 'answer-book' y. We call this our delta  layer $\delta$

3) See how bad we did for our hidden layers. We obviously can't compare our hidden layers to y, because they're not based on the final output. Instead, we'll have to "backtrack" to figure out our margin of error. Think of it in this scenario: "We see our margin of error for our output layer by comparing their answers to our answer-book 'y'. We get the differences here. If the differences are extremely small, then our margin of error for our output is small. This means our previous neurons did well. If we have some neurons with a large margin of error, then our previous neurons did poorly. The only thing to blame right now must be the previous neuron(s) that gave this bad neuron its bad answer. So we must penalize those previous neurons.". In order to penalize them, we let the previous neurons know how bad THEY did by showing them the margin of error we got based on their inputs to us ('us' being the output layer). We share with them how bad we did by multiplying their weights by our $\delta$ value (our margin of error). This is all multiplied by the derivative of the sigmoid function (we'll define this later). We keep doing this process ONLY up until the last hidden layer, NOT the input layer (therefore, this will be layer 2)

$$ \delta^{(l)} = (\Theta^{(2)})^T \delta^{(3)} \cdot g'(z^{(2)}) $$

Where $ g'(z^{(2)}) $ is the derivative of the sigmoid function, defined as:

$$ g'(z) = a^{(l)} \cdot (1 - a^{(l)})$$

4) Now we accumulate our margins of error into one big matrix for each layer. We call this the delta accumulator, $\Delta$. For each layer, it gets our calculated margin of error vector for said vector and multiplies it by our activation layer. We do this from each layer starting with our input layer (layer 1) up until the last hidden layer (left-to-right), which is layer N-1.

$$\Delta_{i,j}^{(l)} = \Delta_{i,j}^{(l)} + a_{j}^{(l)}\delta_{i}^{(l + 1)}$$

Once we get the delta accumulator layers (just 2 in this case), we average it out with all of our training examples. This becomes derivatives of the cost with respect to $\Theta$ for each layer.

$$\frac{\delta}{\delta\Theta_{i,j}^{(l)}}J(\Theta) = \frac{1}{m}\Delta_{i,j}^{(l)}, \ if \ j = 0 $$

$$\frac{\delta}{\delta\Theta_{i,j}^{(l)}}J(\Theta) = \frac{1}{m}(\Delta_{i,j}^{(l)} + \lambda\Theta_{i,j}^{(l)}), \ if \ j \neq 0 $$

You'd rename this J value as theta1 and theta2, to represent an update of their values.

We'll combine our cost function and our backprop into one function just to represent what's happening better (explained above)

First, let's define the derivative of the sigmoid function. We'll call this *sigmoid_gradient)

In [34]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [54]:
def backprop(theta_params, input_size, hidden_size, num_labels, X, y, reg_lambda):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
        
    # (25, 401)
    theta1 = np.matrix(np.reshape(theta_params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    # (10, 26)
    theta2 = np.matrix(np.reshape(theta_params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta_accumulator1 = np.zeros(theta1.shape) # (25, 401)
    delta_accumulator2 = np.zeros(theta2.shape) # (10, 26)
    
    # compute the cost
    J = (1.0 / m) * (np.trace(-y.T * np.log(h)) - np.trace((1 - y).T * np.log(1 - h)))
    
    # add cost regularization term
    J += (float(reg_lambda) / (2 * m)) * (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2)))
    
    ### end of cost function, now on to backprop ###
    for t in range(m):
        # Part 1: run through forwardprop
        a1t = a1[t, :] # (1, 401)
        z2t = z2[t, :] # (1, 25)
        a2t = a2[t, :] # (1, 26)
        ht = h[t, :]   # (1, 10)
        yt = y[t, :]   # (1, 10)
        
        # Part 2: Get delta for layer 3 by seeing how bad we did
        d3t = ht - yt # (1, 10)
        
        # Part 3: Get deltas for hidden layer (layer 2)
        # REMEMBER: np.multiply is element-wise multiplication, while
        # * is matrix multiplication
        z2t = np.insert(z2t, 0, values=np.ones(1)) # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t)) # (1, 26)
        
        # Part 4: Accumulate Deltas for margin of errors into Delta accumulators
        # we'll get an average of our accumulator for all training examples
        # afterward by dividing it by m
        delta_accumulator1 = delta_accumulator1 + (d2t[:, 1:]).T * a1t
        delta_accumulator2 = delta_accumulator2 + d3t.T * a2t
        
    delta_accumulator1 = delta_accumulator1 / m
    delta_accumulator2 = delta_accumulator2 / m
    
    # add the gradient regularization term (the /m is here again because we didn't
    # average out the reg term as well)
    delta_accumulator1[:, 1:] = delta_accumulator1[:, 1:] + (theta1[:, 1:] * reg_lambda) / m
    delta_accumulator2[:, 1:] = delta_accumulator2[:, 1:] + (theta2[:, 1:] * reg_lambda) / m
        
    # unroll the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta_accumulator1), np.ravel(delta_accumulator2)))
    
    return J, grad


Now let's test it out to make sure the function returns what we expect

In [55]:
J, grad = backprop(theta_params, input_size, hidden_size, num_labels, X, y_onehot, reg_lambda)
J, grad.shape

(6.7739154702005706, (10285,))

Now we're ready to train our network and use it to make predictions. This is similay to the previous exercise with multi-class regression

In [57]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=theta_params, args=(input_size, hidden_size, num_labels, X, y_onehot, reg_lambda),  
                method='TNC', jac=True, options={'maxiter': 250})
print "Done:", fmin

Done:   status: 3
 success: False
    nfev: 250
     fun: 0.32842875596276927
       x: array([ -1.31062744e+00,  -8.40792370e-05,  -1.53053993e-04, ...,
        -4.75516368e-01,  -2.98868959e+00,  -5.02640103e+00])
 message: 'Max. number of function evaluations reached'
     jac: array([  4.54561845e-04,  -1.68158474e-08,  -3.06107987e-08, ...,
        -8.97972806e-05,   9.84707324e-05,   1.55790889e-04])
     nit: 20


We put a bound on the number of iterations since the objective function isn't likely to completely converge. Our total cost has dropped below 0.5 though, so that's a good indicator that the algorithm is working

Let's use the theta parameters found to run out forward propagation and get some predictions

In [58]:
X = np.matrix(X)

# we have to reshape the output from the optimizer to match the
# theta parameter matrix shapes that our network is expecting

# (25, 401)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  

# (10, 26)
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ..., 
       [ 9],
       [ 9],
       [ 9]])

Finally, we can compute the accuracy to see how well our trained network is doing

In [59]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))

print 'accuracy = {0}%'.format(accuracy * 100)

accuracy = 99.34%
